In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import cavsim
from cavsim import Measure
from cavsim.connectors import BaseConnector, Connector
from cavsim.channels import ImportChannel, ExportChannel
from cavsim.components import BaseComponent, Component
from cavsim.solvers import BaseSolver, SimpleSolver
from cavsim.fluids import Fluid

In [3]:
from cavsim.pipes.pipe import Pipe
from cavsim.boundaries.left_boundary_pressure import LeftBoundaryPressure
from cavsim.boundaries.left_boundary_velocity import LeftBoundaryVelocity
from cavsim.boundaries.right_boundary_pressure import RightBoundaryPressure
from cavsim.boundaries.right_boundary_velocity import RightBoundaryVelocity
from cavsim.boundaries.simple_pipe_connector import PipeConnectorSimple
from cavsim.boundaries.zeta_joint import ZetaJoint
from cavsim.boundaries.simple_t_joint import SimpleTJoint
from cavsim.boundaries.simple_right_gasdampener import RightGasBubbleSimple
from cavsim.boundaries.frictionless_gasdampener import FrictionlessGasDampener
from cavsim.boundaries.gasdampener import GasDampener
from cavsim.boundaries.pump_suction_valve import PumpSuctionValve

In [4]:
fluid = Fluid(1000, 1e-3, 2.08e9, 2.3e-3,initial_pressure=128000)

In [5]:
def pump(time):
    
    r = 0.02
    lam = 0.0187
    d = 0.07
    #rpm = 120
    rpm = 90
    omega = rpm / 60 * 2* np.pi
    
    velocity = r * omega * (np.sin(omega * time) + lam / 2.0 * np.sin(2 * omega * time))
    result = velocity
    
    return result

In [6]:
def step(time):
    return 0.35 * (time > 0.25)

In [7]:
plogs = []
vlogs = []
flogs = []
rblogs = []
relogs = []
def logging():
    plogs.append(pipe.field_wide_slice('pressure', 0) + 0.0)
    vlogs.append(pipe.field_wide_slice('velocity', 0) + 0.0)
    flogs.append(pipe.field_wide_slice('friction_steady', 0) + 0.0)
    rblogs.append(rb._velocity[0,1])
    relogs.append(pipe.field_wide_slice('reynolds', 0) + 0.0)

In [8]:
boundary_pipe = Pipe(1.5, 0.5, 0.002, 2e11, 2e-4, 4)
pipe = Pipe(1.5, 0.5, 0.002, 2e11, 2e-4, 4)
pipex = Pipe(1.5, 1.0, 0.002, 2e11, 2e-4, 4)
pipe1 = Pipe(0.056, 5.0, 0.002, 2e11, 2e-4, 4)
pipe2 = Pipe(0.025, 0.07, 0.002, 2e11, 2e-4, 10, initial_pressure=20e5)
pipe3 = Pipe(0.025, 0.07, 0.002, 2e11, 2e-4, 10, initial_pressure=20e5)
pipe4 = Pipe(0.07, 0.03, 0.002, 2e11, 2e-4, 10, initial_pressure=20e5)
pipe5 = Pipe(0.025, 28.7, 0.002, 2e11, 2e-4, 4, initial_pressure=20e5)

In [9]:
lb1 = LeftBoundaryPressure(128000)
pipe.connect(lb1)

In [10]:
rb = RightBoundaryVelocity(pump)

In [11]:
tjoint = SimpleTJoint()
pipe.connect(tjoint.left)
boundary_pipe.connect(tjoint.right2)
pipex.connect(tjoint.right)

In [12]:
conn = ZetaJoint(4.5)
pipex.connect(conn)
pipe1.connect(conn)

In [13]:
lb2 = RightGasBubbleSimple(pressure0 = 128000, volume0 = 1.5, polytropic_exponent = 1.1)
lb2.connect(boundary_pipe)

In [14]:
#conn = ZetaJoint(1.0)
#pipe.connect(conn)
#pipe1.connect(conn)

In [15]:
angle = np.pi / 2.0
valve = PumpSuctionValve(7950.0, 3.438, 3438.0, 0.022, 0.068, 0.040, 0.03644173, angle, 7.63e-3, 6.480, 107.00, 74.00, 1.40, -2.40, 0.50, 0.0, 0.0, 25.0e-3)
valve.connect(pipe1)
valve.connect(pipe2)

In [16]:
pump_joint = SimpleTJoint()
pipe2.connect(pump_joint.left)
pipe3.connect(pump_joint.right2)
pipe4.connect(pump_joint.right)

In [17]:
rb.connect(pipe4)

In [18]:
discharge = PumpSuctionValve(7950.0, 3.438, 3438.0, 0.022, 0.068, 0.040, 0.03644173, angle, 7.63e-3, 6.480, 107.00, 74.00, 1.40, -2.40, 0.50, 0.0, 0.0, 25.0e-3)
discharge.connect(pipe3)
discharge.connect(pipe5)

In [19]:
rb2 = RightBoundaryPressure(20e5)

In [20]:
pipe5.connect(rb2)

In [21]:
plogs1 = []
plogs2 = []
plogs3 = []
plogs4 = []
plogs5 = []
plogs6 = []
vlogs = []
vlogs1 = []
flogs = []
rblogs = []
relogs = []
dislog = []
springlog = []
contact_pressurelog = []
velocity = []
dischargedisfieldlog = []


def logging(t):
    plogs.append(pipe.field_wide_slice('pressure', 0) + 0.0)
    dislog.append(valve.field_wide_slice('displacement', 0) + 0.0)
    dischargedisfieldlog.append(discharge.field_wide_slice('displacement', 0) + 0.0)
    springlog.append(valve.field_wide_slice('spring_force', 0) + 0.0)
    contact_pressurelog.append(valve.field_wide_slice('contact_pressure_force', 0) + 0.0)
    plogs1.append(pipe1.field_wide_slice('pressure', 0) + 0.0)
    plogs2.append(pipe2.field_wide_slice('pressure', 0) + 0.0)
    plogs3.append(pipe3.field_wide_slice('pressure', 0) + 0.0)
    plogs4.append(pipe4.field_wide_slice('pressure', 0) + 0.0)
    plogs5.append(pipe5.field_wide_slice('pressure', 0) + 0.0)
    velocity.append(valve.field_wide_slice('valve_velocity', 1) + 0.0)

In [22]:
solver = SimpleSolver()
solver.fluid = fluid
solver.seeds = [pipe1, pipe2, pipe3, pipe4, pipe5]
solver._callback = logging

In [ ]:
solver.solve(1e20, 1.0, 3)

 0:00:00 [                              |  0.00%]  2:17:30  Currently at time   0.000 of   1.000

l:\cavsim\cavsim\base\solvers\simple_solver.py:46: UserWarning: Smaller timestep required by component! (0.000787889098013947 < 1e+20 by <cavsim.pipes.pipe.Pipe object at 0x000002905A5A0550>)
  warn('Smaller timestep required by component! ({} < {} by {})'.format(component_time, delta_t, component))
l:\cavsim\cavsim\base\solvers\simple_solver.py:46: UserWarning: Smaller timestep required by component! (0.00041137667560372116 < 1e+20 by <cavsim.pipes.pipe.Pipe object at 0x000002905A5A9748>)
  warn('Smaller timestep required by component! ({} < {} by {})'.format(component_time, delta_t, component))
l:\cavsim\cavsim\base\solvers\simple_solver.py:46: UserWarning: Smaller timestep required by component! (4.690432701998517e-06 < 1e+20 by <cavsim.pipes.pipe.Pipe object at 0x000002905A5A5358>)
  warn('Smaller timestep required by component! ({} < {} by {})'.format(component_time, delta_t, component))
l:\cavsim\cavsim\base\solvers\simple_solver.py:46: UserWarning: Smaller timestep required by c

 1:30:11 [==============                | 49.58%]  1:31:42  Currently at time   0.496 of   1.000

In [ ]:
pfield1 = np.stack(plogs1)
#pfield2 = np.stack(plogs2)
#pfield3 = np.stack(plogs3)
pfield4 = np.stack(plogs4)
#pfield5 = np.stack(plogs5)
disfield = np.stack(dislog)
#velfield = np.stack(velocity)
disdisfield = np.stack(dischargedisfieldlog)

In [ ]:
time = np.linspace(0.0, 1.5, int(pfield4[:,-2].shape[0]))
#plt.figure(figsize=(16, 10))
fig, ax1 = plt.subplots(figsize=(16, 10))
#plt.subplot(1, 2, 1)
#plt.plot(vfield[:,0])
#plt.plot(1, 2, 2)
#val1 = 157500
#val = -220000
val = -1
val1 = 0

#upper_force = upperp_field[val1:val, 0] + gfield[val1:val, 0] + springforce[val1:val, 0]
#lower_force = lowp_field[val1:val, 0] + contact_pressure[val1:val, 0]
#result1 = lower_force - upper_force
#result2 = flow_field[val1:val, 0] - gfield[val1:val, 0] - springforce[val1:val, 0] - dampingfield[val1:val, 0]
#upper_flow = gfield[val1:val, 0] + springforce[val1:val, 0] + dampingfield[val1:val, 0]
#lower_flow = flow_field[val1:val, 0]
#ax1.plot(time[val1:val], upper_flow, color='red')
#ax1.plot(time[val1:val], lower_flow, color='blue')


#plt.subplot(1, 2, 2)
#ax1.plot(result1[val1:val])
#ax1.plot(result2[val1:val])
#ax1.plot(time[val1:val], upperp_field[val1:val, 0], color='red')
#ax1.plot(time[val1:val], lowp_field[val1:val, 0], color='green')
#ax1.plot(time[val1:val], dampingfield[val1:val, 0], color='blue')

#ax1.plot(time[val1:val], contact_pressure[val1:val, 0], color='black')
#ax1.plot(time[val1:val], springforce[val1:val, 0], color='grey')
#ax1.plot(time[val1:val], upper_force, color='red')
#ax1.plot(time[val1:val], lower_force, color='blue')

#plt.plot(time[val1:val], pfield2[val1:val, 1], color='red')
#plt.plot(time[val1:val], pfield1[val1:val, -2])


#ax1.plot(time[val1:val], deltap[val1:val, 0], color='green')

#plt.xlim(time[val1],time[val])
plt.ylim(0.0e5, 5e5)

color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('Pressure', color=color)
#ax1.set_ylim(-0e5, 6e5)
#ax1.plot(time[val1:val], pfield3[val1:val, 1], color='red')
ax1.plot(time[val1:val], pfield4[val1:val, -2], color='green')
ax1.plot(time[val1:val], pfield1[val1:val, -2], color='black')
ax1.tick_params(axis='y', labelcolor=color)


ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.plot(time[val1:val], disfield[val1:val, 0]*1e3, color=color)  # we already handled the x-label with ax1
ax2.plot(time[val1:val], disdisfield[val1:val, 0]*1e3, color='black')
#ax2.plot(time[val1:val], velfield[val1:val, 0], color = 'pink')
#ax2.plot(time[val1:val], vfield[val1:val, 1], color = 'red')
#ax2.plot(time[val1:val], zeta[val1:val, 0], color='green')
#ax2.plot(time[val1:val], flow_field[val1:val, 0], color='black')
ax2.tick_params(axis='y', labelcolor=color)

#plt.subplot(1, 2, 1)
#plt.plot(time[val1:val], disfield[val1:val, 0]*1000, color='black')

#plt.plot(time[val1:val], pump(time[val1:val]))
#plt.plot(time[val1:val], velfield[val1:val, 0], color='blue')
#plt.plot(time[val1:val], pfield2[val1:val, 1], color='red')
#pltplot(time[val1:val], pfield1[val1:val, -2], color='green')
#plt.ylim(0.0e5, 8.0e5)
fig.tight_layout()
plt.show()
#print(np.shape(pfield7[val1:val, 1]))